### Heat map visualization

In [ ]:
# Set parameter file and model to load
from utils.myUtils import mostRecent 

# chose an existing pickledParamFile and run loadModelForEvaluation
pickledParamFile = mostRecent("demoModel/*modelParams.pkl")
#pickledParamFile = mostRecent("/home/lonce/ZCODE/PYTORCH/RNNControl/model/*modelParams.pkl")

# choose saved model to run (defaults to the last saved if loadmodelfile is undefined or None)
# Note: looks in same directory as pickledParamFile
loadModelFileName=None
#loadModelFileName='2018-11-23_09-37-53_model_epoch1_step90.pth'

In [ ]:
# run notebook to load analysis/synthesis code
%run loadModelForEvaluation.ipynb

if 'sample_rate' not in params :   #older versions didn't save sample_rate
    params['sample_rate']=16000

### Primer

In [ ]:
# Make noisy nuetral Priming signal 
myp=np.zeros([1,seqLen,len(params['props'])+1])
# !! PRIMING DOES MATTER - has to be a certain level of noise
myp[0,:,0]=.3*np.random.ranf([seqLen])-.15 #signal
myp[0,:,1]= .45+.1*np.random.ranf([seqLen])     #instrument
myp[0,:,2]= .5*np.random.ranf([seqLen])    #pressure1
#myp[0,:,3]=1-myp[0,:,2]    #pressure2
myp[0,:,3]= np.random.ranf([seqLen])    #pitch
myprimer=torch.tensor(myp, dtype=torch.float)

### <font color="brown"> Generate a signal </font>

In [ ]:
from ipywidgets import interact,interactive
from ipywidgets import FloatSlider, IntSlider
from IPython.display import Audio

genlen= int(.6*sr) #not including priming

gi=0
gv=0
gp=0


def fsweep(snum) :   #nsnum is sample number not including priming
    
    t=snum/genlen #snum/params['sample_rate']
    if (t<0.1) :
        v=0
    elif t<.6 :
        v=gv
    else :
        v=0
    return np.array([gi,v, gp]) 

#global var for the hidden nodes returned from generate() inside widgetfunc
hdn=[]


def widgetfunc(x,y,z):
    paramvect = np.array([x,y,z])
    global gi
    global gv
    global gp
    
    global hdn
    
    gi=x
    gv=y
    gp=z
    
    signal, hdn = generate(rnn,genlen,primer=myprimer,paramvect = fsweep, returnHiddenSequence=True)
    plt.figure(figsize=(20,3))
    #lot_signal(signal[seqLen:max_length])
    plt.plot(np.arange(len(signal[seqLen:])), signal[seqLen:]) 
    plt.grid()
    plt.show()
    display(Audio(data=signal[0:],rate=sr))
    return signal
    
xslider=FloatSlider(min=0, max=1, step=.01, value=1.0, continuous_update=False, description='instID')
yslider=FloatSlider(min=0, max=1, step=.01, value=.8, continuous_update=False, description='pressure1')
zslider=FloatSlider(min=0, max=1, step=.01, value=0.5,  continuous_update=False, description='midiPitch')

w = interactive(widgetfunc, x=xslider, y=yslider, z=zslider)
display(w)


In [ ]:
# Heatmap for signal from last call to generate()

import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl

#Separate layers for visualization 
#want each layers[n] to be 40 rows of length timesteps=400 (40 x 400)
layers = [np.transpose([x[n] for x in hdn]) for n in range(0,4) ]

t=list(range(0,len(hdn)))
elmts=[list(range(0,len(layers[n][0]))) for n in range(0,4)]

print("Layer[0] has {} rows of length {}".format(len(layers[0]), len(layers[0][0])))

#*************************
from ipywidgets import interact,interactive
from ipywidgets import FloatSlider, IntSlider
from IPython.display import Audio

def widgetfunc(layer,unit,snum,scale):

    global t
    global layers
    
    
    endSnum=int(min(snum+40*math.pow(2,scale) , len(t)-1))
    print("endSnum is {}".format(endSnum))
    
    numxticks=10
    xtick_spacing=max(int((endSnum-snum)/numxticks), 1)
    xlabels=range(snum, endSnum, xtick_spacing)
    
    aspectRatio=(endSnum-snum)/40

    
    data = layers[layer][unit:unit+40,snum:endSnum]
    print("data.shape = {}".format(data.shape))
    
    fig = plt.figure()
    fig, ax = plt.subplots(1,1, figsize=(30,30))

    heatplot = ax.imshow(data, cmap='RdYlBu',  vmin=0, vmax=1)
    ax.set_xticklabels(xlabels)
    ax.set_yticklabels(elmts[layer][unit:unit+40])


    ytick_spacing = 1
    #ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(xtick_spacing))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(ytick_spacing))

    ax.set_title("Heatmap of hidden unit layer {}".format(layer))
    ax.set_xlabel('sample number')
    ax.set_ylabel('elmt')
    
    ax.set_aspect(aspectRatio)
    plt.rcParams.update({'font.size': 26})
    
    
layerSlider=IntSlider(min=0, max=3, step=1, value=0, continuous_update=False, description='layer')
unitSlider=IntSlider(min=0, max=40, step=1, value=0, continuous_update=False, description='unit')
snumSlider=IntSlider(min=0, max=len(t), step=1, value=0,  continuous_update=False, description='sampleNumber')
scaleSlider=FloatSlider(min=0, max=12, step=.1, value=12,  continuous_update=False, description='scale')

w = interactive(widgetfunc, layer=layerSlider, unit=unitSlider, snum=snumSlider, scale=scaleSlider)
display(w)


In [ ]:
import matplotlib as mpl

def widgetfunc(layer,snum,scale):

    global t
    global layers
    
    print("length of time array = {}".format(len(t)))
    
    endSnum=int(min(snum+40*math.pow(2,scale) , len(t)-1))
    print("endSnum is {}".format(endSnum))
    
    numxticks=10
    xtick_spacing=max(int((endSnum-snum)/numxticks), 1)
    xlabels=range(snum, endSnum, xtick_spacing)
    
    aspectRatio=(endSnum-snum)/40

    
    numplots=40
    f, axarr = plt.subplots(numplots, sharex=True, sharey=True, figsize=(30,30))
    for i in range(numplots) :
        axarr[i].plot(t[snum:endSnum], layers[layer][i,snum:endSnum])
        axarr.flat[i].set(xlabel='samples', ylabel=i)
        axarr[i].set_yticks([])

    
    axarr[0].set_title('Activations for layer {}'.format(layer))

    # Fine-tune figure; make subplots close to each other and hide x ticks for
    # all but bottom plot.
    f.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
    

    
layerSlider=IntSlider(min=0, max=3, step=1, value=0, continuous_update=False, description='layer')
snumSlider=IntSlider(min=0, max=len(t), step=1, value=0,  continuous_update=False, description='sampleNumber')
scaleSlider=FloatSlider(min=0, max=12, step=.1, value=12,  continuous_update=False, description='scale')

w = interactive(widgetfunc, layer=layerSlider,  snum=snumSlider, scale=scaleSlider)
display(w)